There are some repetitivetweets. This is because one person repeats a tweet (or very similar tweets that)many times in some days for emphasis or other reasons like reporting weather.

*Clustering for sentence embeddingworks good when we have few words in the sentence (like: fire tonight)

*Sentence embedding works wellwith specific words in the sentence (like Australia and their cities) but itdoes not mean that all tweets about Australia are in one cluster.

*Sentence embedding works goodfor sentences with many similar words in each tweet.

*Stemming (which we use incleaning) is not good for some words. For example, for “business” the stemmingalgorithm gives “busi” while the embedding algorithm does not recognize thisword and consider it as bus+i. Therefore, although on one hand, stemming convertsdifferent derivatives of one word to one word, on the other hand, maybe theembedding algorithm does not recognize that word so the embedding for that wordis not much semantic. As I studied this problem exists in some stemming algorithms.

*Regarding fast algorithmsfor big data clustering, usually kmeans is the fastest (some implementations ofit have time complexity of O(n). But the problem is that it needs a number ofclusters in advance. Then dbscan clustering with time complexity O(n * log n)has been used too. Hierarchical clustering for big data has also been used butit has this problem we do not know where to cut the from the top and bottom ofthe dendrogram. (It should be done manually).

*For the mng set, first Iused optics clustering algorithm but it did not finish. I used dbscan forclustering. With bdscan, which needs one more parameter that needs knowledge ofdata, the time for clustering was different. The code was run for threedifferent pairs of parameters. Nevertheless, the result is not good for all ofthem since most of the data are in the noise cluster or in one cluster. One time itdid not finish even after 1 day. The next time it finished after 5.5 hours withone pair of parameters (87 clusters + 1 noise cluster). Almost all of the tweetsclustered as noise. I changed the parameters. It finished after 7.5 hours (100cluster + 1 noise cluster), about most of the tweets were in one cluster and most ofthe remaining were in noise clusters. With other parameters, it took 5 mins! (271clusters + noise cluster). But again, almost all of the tweets were clusteredas noise. So, for all three cases, there are many tweets in the noise cluster ormany tweets in one cluster. Other clusters have usually less than 10 tweets,mostly around 5 tweets and usually none of them were related to management. It meansall related tweets are in cluster 0 which is useless.  

*For ent set, I used kmeanswith 50, 100 and 1000 clusters. Number of members in each cluster is morerational. One way to find beliefs is to find clusters that have most usage ofentrepreneur. Then, frequent words in these clusters can be related to beliefs.This method seems good if each tweet is reduced to some few words and theembedding is fully semantic. Of course, it does not mean that clusters withoutthis word are not relevant, but for first try let us assume this. Since we mayhave many words in each tweet and the embedding is not very semantic, in almostall of the clusters we have this word for three cases (50 , 100, 1000clusters). For 1000 clusters, I found 20 most frequent words from 5 clustersthat have the most usage of entrepreneur. As it is seen from the result, somewords are not related, some can be beliefs like tech (technology), invest(investment), innov (innovation),..... Some words cannot be meaningful alonelike human or learn. Finding clustering by words is not complete (even if wesuppose that embedding is 100% semantic which is not). For example, if we havea tweet about “customer satisfaction is important”. By separate word clusteringamong all words of all tweets, we may find just “customer” which is useless orwe may find “satisfaction” which is incomplete: satisfaction of what?customers, employees, …. so, working on words individually does not seemperfect although for the first try it is good.

*Regarding different timefor dbscan and kmeans (although the number of data is fixed), I think the timeof these algorithms can be dependent on other parameters and also it is relatedto my laptop that is running some codes simultaneously, which makes my laptop slow. 

*We need reducingunnecessary words, and getting abstract features in the sentences to havebetter results. 

In [124]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import pickle
import time
import numpy as np
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
from sklearn.cluster import OPTICS
import time 
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import sys
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jafar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
Stem=stemmer.stem

In [119]:
print(Stem('entrepreneur'))
print(Stem('manager'))

entrepreneur
manag


In [3]:
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
cur.execute("SELECT tweet from ent_2019_100K limit 100000 ")
rows_ent = cur.fetchall()
# there is no repetative tweet
con.close()

print(len(rows_ent))

Database opened successfully
100000


In [4]:
print(type(rows_ent))
rows_ent[0]

<class 'list'>


('These 8 simple things will make you the best https://t.co/kIAQsodgb0 by @RoyOsing #niche #education #business https://t.co/2yKiwiikmB',)

In [5]:
tweet_ent=[]
for row in rows_ent:
#    print (row)
    tweet_ent.append(row[0])

In [6]:
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
cur.execute("SELECT tweet from mng_2019_100K limit 100000")
rows_mng = cur.fetchall()
# there is no repetative tweet
con.close()

print(len(rows_mng))

Database opened successfully
100000


In [7]:
tweet_mng=[]
for row in rows_mng:
#    print (row)
    tweet_mng.append(row[0])

In [491]:
with open("tweet_mng.txt", "wb") as fp:   #Pickling
  pickle.dump(tweet_mng, fp)

with open("tweet_ent.txt", "wb") as fp:   #Pickling
  pickle.dump(tweet_ent, fp)
 

In [11]:
  
with open("tweet_ent.txt", "rb") as fp:   # Unpickling
  tweet_ent = pickle.load(fp)
  
with open("tweet_mng.txt", "rb") as fp:   # Unpickling
  tweet_mng = pickle.load(fp)

In [33]:
print(tweet_mng[0:10])
print(tweet_ent[0:10])

['���� no words needed! https://t.co/Q2triMafff', 'RT @grynbaum: Chuck Todd opened his MSNBC show today by saying: "Let\'s be frank: a national nightmare is upon us. The basic rules of our de…', 'RT @PFF_College: Oregon’s CJ Verdell made defenders miss all season https://t.co/5M9PmMIu8f', 'Great to see young @NewryCityAFC striker John McGovern rewarded for his performances this season with a starting appearance for @OfficialIrishFA U19s tonight against Norway hopefully the first of many https://t.co/onxfTUUVlV', 'RT @naastynun: every single day is the same day just worse https://t.co/t6sQXYP35u', "Nah, you ain't it.", '@MtyandaSipho I can’t even get @DanGilchrist94 out for a night shift he wants to much https://t.co/tRMPF5RtTT', '@PiyushGoyal Great, although wished it came a decade earlier', 'Anon I want!! https://t.co/rvGAWeLqka', '@MooSnuckel Nothing man, gamers rise up!']
['These 8 simple things will make you the best https://t.co/kIAQsodgb0 by @RoyOsing #niche #education #business 

In [20]:
def cleaning (text):
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    #removing emoji
    text = emoji_pattern.sub(r' ', text)   

    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)

    #removing some special charachter  
    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`;•«,@:~!=%&]+", ' ',text) # except ' ’
    
    #removing hashtag
    text=re.sub('#', '', text) 
    
    #removing numbers
    text=re.sub("(\d+)",' ',text)
#    print(text)
    #removing numbers (not attached)
#    text=re.sub(r"(^|\s)-?(\d+)\.?(\d*)(\s)", '', text)
#    text=re.sub("(^||\d+)\.(\d+) ",'',text)
#    text=re.sub("[0-9+]\.?(\d+)(\.)",' ',text)
#    text=re.sub("[0-9+]\.?(\d+)($)",' ',text)
#    print(text)
    
#    return text

#def lower_case (text):
    text = text.lower() 
#    print(text)
#    return text_lower

#def tokenization (text):
    text= nltk.word_tokenize(text)
#    print (text)
#    return text_tokens

#def removing_stopwords (text):
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()]
#    text = [word for word in text if word not in stopwords.words()]
    text = [word for word in text if word not in cachedStopWords]
#    print(text)
#    return text_without_sw

#def stemming (text):
    #stemmer = SnowballStemmer("english")
    #text_stems=stemmer.stem(tokens_without_sw)
#    text = [stemmer.stem(word) for word in text]
    text = [Stem(word) for word in text]

#    print(text)
    
    text=' '.join(text)
#    print(text)
#    print(text,'\n')   
    
    return text

In [45]:
print(tweet_ent[0:10])
print()
print(cleaning(tweet_ent[6]))
cleaning('i lion\tiger\cat')

['These 8 simple things will make you the best https://t.co/kIAQsodgb0 by @RoyOsing #niche #education #business https://t.co/2yKiwiikmB', '@wisdomjack22 @dat_cutiee @ogundamisi Absolutely!', 'Advancing the human condition through food and beverage https://t.co/OHI9ero462', 'Watched the premier of  #mixedish tonite.  Love the cast!! Bow’s perspective and storytelling of growing up biracial in the 80s is heartfelt. Hope there is lots of 80s music. @mixedishabc @TraceeEllisRoss https://t.co/HyroAgyFkw', 'Yessir! We LIT LIT! https://t.co/WvRzew6r3P', 'RT @AOC: I’m not going to quote the NY Post’s horrifying, hateful cover.\n\nHere’s 1 fact: @IlhanMN is a cosponsor of the 9/11 Victim Compens…', 'RT @CJ_Johnson17th: Things Worth Discussing Every Day:\n\nLeadership\nTime\nFamily\nDevelopment\nFitness\nMoney\nThe Future\n\nThings Not Worth Discu…', 'RT @lilhoneybabee: me thinking about all the shit i didn’t deserve to go through this whole entire year https://t.co/E9Z7FBW1bR', 'In the pool, we

'lion iger cat'

In [493]:
len(tweet_mng)

100000

In [47]:
t0 = time.time()
tweet_mng_cleaned_full=[]

for tweet in tweet_mng:
    tweet_cleaned=cleaning(tweet)
    if tweet_cleaned!='': # or if not text_str
        tweet_mng_cleaned_full.append(tweet_cleaned)
        
print( time.time() - t0)

47.14022779464722


In [48]:
t0 = time.time()
tweet_ent_cleaned_full=[]

for tweet in tweet_ent:
    tweet_cleaned=cleaning(tweet)
    if tweet_cleaned!='': # or if not text_str
        tweet_ent_cleaned_full.append(tweet_cleaned)
        
print( time.time() - t0)

29.648709297180176


In [50]:
with open("tweet_mng_cleaned_full.txt", "wb") as fp:   #Pickling
  pickle.dump(tweet_mng_cleaned_full, fp)

with open("tweet_ent_cleaned_full.txt", "wb") as fp:   #Pickling
  pickle.dump(tweet_ent_cleaned_full, fp)

In [36]:
t0 = time.time()
tweet_mng_cleaned=[]

for tweet in tweet_mng:
    tweet_cleaned=cleaning(tweet)
    if tweet_cleaned!='': # or if not text_str
        tweet_mng_cleaned.append(tweet_cleaned)
        
print( time.time() - t0)

12.177435398101807


In [37]:
len(tweet_mng_cleaned)

99998

In [38]:
t0 = time.time()
tweet_ent_cleaned=[]

for tweet in tweet_ent:
    tweet_cleaned=cleaning(tweet)
    if tweet_cleaned!='': # or if not text_str
        tweet_ent_cleaned.append(tweet_cleaned)
        
print( time.time() - t0)

13.004223823547363


In [39]:
len(tweet_ent_cleaned)

99999

In [41]:
with open("tweet_mng_cleaned.txt", "wb") as fp:   #Pickling
  pickle.dump(tweet_mng_cleaned, fp)

with open("tweet_ent_cleaned.txt", "wb") as fp:   #Pickling
  pickle.dump(tweet_ent_cleaned, fp)
 

In [42]:
  
with open("tweet_ent_cleaned.txt", "rb") as fp:   # Unpickling
  tweet_ent_cleaned_retrieved = pickle.load(fp)
  
with open("tweet_mng_cleaned.txt", "rb") as fp:   # Unpickling
  tweet_mng_cleaned_retrieved = pickle.load(fp)

In [49]:
print(tweet_mng_cleaned[0:10])
print(tweet_mng_cleaned_retrieved[0:10])

['no words needed', "rt chuck todd opened his msnbc show today by saying let 's be frank a national nightmare is upon us the basic rules of our de…", 'rt oregon ’ s cj verdell made defenders miss all season', 'great to see young striker john mcgovern rewarded for his performances this season with a starting appearance for u s tonight against norway hopefully the first of many', 'rt every single day is the same day just worse', "nah you ai n't it", 'i can ’ t even get out for a night shift he wants to much', 'great although wished it came a decade earlier', 'anon i want', 'nothing man gamers rise up']
['no words needed', "rt chuck todd opened his msnbc show today by saying let 's be frank a national nightmare is upon us the basic rules of our de…", 'rt oregon ’ s cj verdell made defenders miss all season', 'great to see young striker john mcgovern rewarded for his performances this season with a starting appearance for u s tonight against norway hopefully the first of many', 'rt every s

In [13]:
'''
with open("test.txt", "w") as file:
    file.write(str(tweet_mng))

with open("test.txt", "r") as file:
    data2 = eval(file.readline())

# Let's see if data and types are same.
print(tweet_mng, type(tweet_mng), type(tweet_mng[0]))
print(data2, type(data2), type(data2[0]))
'''

'\nwith open("test.txt", "w") as file:\n    file.write(str(tweet_mng))\n\nwith open("test.txt", "r") as file:\n    data2 = eval(file.readline())\n\n# Let\'s see if data and types are same.\nprint(tweet_mng, type(tweet_mng), type(tweet_mng[0]))\nprint(data2, type(data2), type(data2[0]))\n'

In [531]:
'''
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_1 = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
marked_text = "[CLS] " + text + " [SEP]"
tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
with torch.no_grad():
    outputs = model_1(tokens_tensor, segments_tensors)
hidden_states = outputs[1][0]
sentence_vec=hidden_states
'''

'\nfrom transformers import BertTokenizer, BertModel\ntokenizer = BertTokenizer.from_pretrained(\'bert-base-uncased\')\nmodel_1 = BertModel.from_pretrained(\'bert-base-uncased\',\n                                  output_hidden_states = True, # Whether the model returns all hidden-states.\n                                  )\nmarked_text = "[CLS] " + text + " [SEP]"\ntokenized_text = tokenizer.tokenize(marked_text)\nindexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)\nsegments_ids = [1] * len(tokenized_text)\ntokens_tensor = torch.tensor([indexed_tokens])\nsegments_tensors = torch.tensor([segments_ids])\nwith torch.no_grad():\n    outputs = model_1(tokens_tensor, segments_tensors)\nhidden_states = outputs[1][0]\nsentence_vec=hidden_states\n'

In [505]:
t0 = time.time()
mng_embedding1=[]
for tweet in tweet_mng_cleaned:
    mng_embedding1.append(model.encode(tweet))
print( time.time() - t0)

25872.352993249893


In [503]:
t0 = time.time()
mng_embedding = model.encode(tweet_mng_cleaned)
print( time.time() - t0)

11408.33846950531


In [504]:
t0 = time.time()
ent_embedding = model.encode(tweet_ent_cleaned)
print( time.time() - t0)

12073.187916517258


In [506]:
with open("tweet_mng_clened_embedding.txt", "wb") as fp:   #Pickling
  pickle.dump(mng_embedding, fp)

with open("tweet_ent_cleaned_embedding.txt", "wb") as fp:   #Pickling
  pickle.dump(ent_embedding, fp)

In [532]:
(mng_embedding[0]-mng_embedding1[0])/mng_embedding[0] # so the result is nearly the same but the spped is different
type(mng_embedding[1]) #numpy

numpy.ndarray

In [82]:
with open("tweet_ent_cleaned_full.txt", "rb") as fp:   # Unpickling
  tweet_ent_cleaned_full = pickle.load(fp)
  
with open("tweet_mng_cleaned_full.txt", "rb") as fp:   # Unpickling
  tweet_mng_cleaned_full = pickle.load(fp)

In [54]:
t0 = time.time()
mng_full_embedding = model.encode(tweet_mng_cleaned_full)
print( time.time() - t0)

14113.82386136055


In [67]:
t0 = time.time()
ent_full_embedding = model.encode(tweet_ent_cleaned_full)
print( time.time() - t0)

8145.297198295593


In [57]:
with open("tweet_mng_cleaned_full_embedding.txt", "wb") as fp:   #Pickling
  pickle.dump(mng_full_embedding, fp)

In [68]:
with open("tweet_ent_cleaned_full_embedding.txt", "wb") as fp:   #Pickling
  pickle.dump(ent_full_embedding, fp)

In [61]:
tweet_ent_cleaned_full[0]

'simpl thing make best nich educ busi'

In [4]:
with open("tweet_mng_cleaned_full_embedding.txt", "rb") as fp:   # Unpickling
  mng_full_embedding = pickle.load(fp)
  
with open("tweet_ent_cleaned_full_embedding.txt", "rb") as fp:   # Unpickling
  ent_full_embedding = pickle.load(fp)

In [113]:
t0 = time.time()
#ent_full_clustering = OPTICS(min_samples=5).fit(ent_full_embedding[0:1200])
#ent_full_clustering = DBSCAN(eps=3, min_samples=2).fit(ent_full_embedding)
kmeans = KMeans(n_clusters=1000) #760s for 100 clusters , 5699 s for 1000 clusters
ent_full_clustering = kmeans.fit(ent_full_embedding)

print( time.time() - t0)

5698.801934480667


For kmenas clustering with 1000 clusters we have:

Which clusters have keyword "entreprenuer" and number of members in that cluster

In [121]:
ent_kmeans_labels=np.unique(ent_full_clustering.labels_)
for label in ent_kmeans_labels:
#    print('cluster:',label)
    bb=np.where(ent_full_clustering.labels_==label)
#    print('number of members:',len(list(bb[0])))
    ind=list(bb[0])
#    print(ind)
    target=0
    for j in ind:
        if 'entrepreneur' in tweet_ent_cleaned_full[j]:
#            print(tweet_ent_cleaned_full[j])
            target +=1
    if target >=1:
        print('cluster:',label)
        print('number of members:',len(list(bb[0])))
        print("number of targets:",target)    
        print('****'*5)

cluster: 2
number of members: 113
number of targets: 2
********************
cluster: 4
number of members: 117
number of targets: 2
********************
cluster: 9
number of members: 147
number of targets: 1
********************
cluster: 13
number of members: 70
number of targets: 30
********************
cluster: 16
number of members: 107
number of targets: 4
********************
cluster: 17
number of members: 75
number of targets: 1
********************
cluster: 19
number of members: 110
number of targets: 1
********************
cluster: 20
number of members: 51
number of targets: 1
********************
cluster: 21
number of members: 152
number of targets: 1
********************
cluster: 41
number of members: 96
number of targets: 3
********************
cluster: 42
number of members: 125
number of targets: 1
********************
cluster: 43
number of members: 92
number of targets: 1
********************
cluster: 45
number of members: 65
number of targets: 1
********************
cluster

In [134]:
most_ind=[603,614,742,746,759]

One way is to find clusters that have most usage of entrepreneur. Then, frequent words in these clusters can be related to beliefs.  

In [136]:
from nltk import FreqDist

for label in most_ind:
#    print('cluster:',label)
    bb=np.where(ent_full_clustering.labels_==label)
#    print('number of members:',len(list(bb[0])))
    ind=list(bb[0])
#    print(ind)
    words=[]
    for j in ind:
        words.extend(tweet_ent_cleaned_full[j].split())
    fdist = FreqDist(words)
    mostCommon= fdist.most_common(20)
    print(mostCommon)
    print('cluster:',label)
    print('number of members:',len(list(bb[0])))
    print('****'*5)

[('entrepreneur', 43), ('rt', 29), ('success', 25), ('great', 19), ('thank', 16), ('’', 10), ('thing', 10), ('busi', 9), ('entrepreneurship', 9), ("'s", 9), ('entrepreneuri', 7), ('leader', 7), ('support', 7), ('latest', 7), ('way', 6), ('good', 6), ('daili', 6), ('founder', 5), ('look', 5), ('share', 5)]
cluster: 603
number of members: 109
********************
[('entrepreneur', 76), ('success', 43), ('startup', 26), ('busi', 25), ('hardwork', 25), ('coach', 23), ('hustl', 22), ('businessown', 21), ('onlinebusi', 21), ('freedom', 20), ('inspir', 20), ('ambit', 19), ('thinkbig', 18), ('businessman', 18), ('money', 16), ('emprendimiento', 15), ('entrepreneurlif', 14), ('make', 11), ('truth', 11), ('photograph', 11)]
cluster: 614
number of members: 84
********************
[('entrepreneurship', 65), ('entrepreneur', 22), ('busi', 14), ('innov', 10), ('rt', 9), ('via', 9), ('skill', 8), ('entrepreneuri', 7), ('amp', 6), ('market', 6), ('sale', 6), ('human', 6), ('social', 5), ("'s", 5), ('e

As it is seen some words are not related, some can be beliefs like tech (technology), invest (investment,)innov (innovation),....
Some words cannot be meaningful alone like human or learn. 

We need better cleaning method, better stemming method (if exists), reducing unnecessary words, and getting abstract features in the sentences to have better results. 

**Also, as it is seen many clusters have the keyword "entrepreneur" although some tweets may be related to entrepreneurship but do not have this word directly. 
So, extracting most (20 or 50) frequent words from them is not useful.**

For kmeans with 100 clusters we have:

In [112]:
ent_kmeans_labels=np.unique(ent_full_clustering.labels_)

for label in ent_kmeans_labels:
    print('cluster:',label)
    bb=np.where(ent_full_clustering.labels_==label)
    print('number of members:',len(list(bb[0])))
    ind=list(bb[0]) 
#    print(ind)
    target=0
    for j in ind:
        if 'entrepreneur' in tweet_ent_cleaned_full[j]:
#            print(tweet_ent_cleaned_full[j])
            target +=1
    print("number of targets:",target)    
    print('****'*5)

cluster: 0
number of members: 979
number of targets: 15
********************
cluster: 1
number of members: 994
number of targets: 0
********************
cluster: 2
number of members: 973
number of targets: 2
********************
cluster: 3
number of members: 815
number of targets: 2
********************
cluster: 4
number of members: 1714
number of targets: 0
********************
cluster: 5
number of members: 1589
number of targets: 3
********************
cluster: 6
number of members: 1071
number of targets: 0
********************
cluster: 7
number of members: 1499
number of targets: 10
********************
cluster: 8
number of members: 698
number of targets: 0
********************
cluster: 9
number of members: 2302
number of targets: 0
********************
cluster: 10
number of members: 1216
number of targets: 0
********************
cluster: 11
number of members: 593
number of targets: 4
********************
cluster: 12
number of members: 579
number of targets: 1
********************
c

Again, for kmeans with 100 clusters,  many clusters have the keyword "entrepreneur" so extracting frequent words (as beliefs) is not meaningful. This is true for kmenas with 50 clusters.

In [87]:
ent_kmeans_labels=np.unique(ent_full_clustering.labels_)
type(ent_kmeans_labels)

numpy.ndarray

In [96]:
for label in ent_kmeans_labels:
    print('cluster:',label)
    bb=np.where(ent_full_clustering.labels_==label)
    print('number of members:',len(list(bb[0])))
    ind=list(bb[0]) 
#    print(ind)
    target=0
    for j in ind:
        if 'entrepreneur' in tweet_ent_cleaned_full[j]:
#            print(tweet_ent_cleaned_full[j])
            target +=1
    print("number of targets:",target)    
    print('****'*5)

cluster: 0
number of members: 2859
number of targets: 9
********************
cluster: 1
number of members: 926
number of targets: 1
********************
cluster: 2
number of members: 1246
number of targets: 0
********************
cluster: 3
number of members: 1585
number of targets: 1
********************
cluster: 4
number of members: 660
number of targets: 6
********************
cluster: 5
number of members: 1491
number of targets: 0
********************
cluster: 6
number of members: 2580
number of targets: 2
********************
cluster: 7
number of members: 2434
number of targets: 48
********************
cluster: 8
number of members: 1317
number of targets: 5
********************
cluster: 9
number of members: 2444
number of targets: 0
********************
cluster: 10
number of members: 1444
number of targets: 2
********************
cluster: 11
number of members: 1982
number of targets: 0
********************
cluster: 12
number of members: 2482
number of targets: 23
*****************

For OPTICS clustering, the clustering did not finish. With DBSCAN, which needs one more parameter than needs knowledge of data, the time for clustering was different. It seems that whatever eps is smaller, the running time is less. The code was run for three different pairs of parameters. Nevertheless, the result is not good for all of them since most of data are in the noise cluster or ine one cluster. So again we extracting frequent words (as an approximation of beliefs) can not be meaningful. 

In [98]:
t0 = time.time()
#mng_full_clustering = OPTICS(min_samples=10).fit(mng_full_embedding)
mng_full_clustering = DBSCAN(eps=0.25, min_samples=5).fit(mng_full_embedding) # 19864s for 3-10, 26922s for 10_5, 345s for 0.25-5

print( time.time() - t0)

345.9003360271454


In [99]:
np.unique(mng_full_clustering.labels_)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 17

In [100]:
with open("tweet_mng_cleaned_full_clustering_dbscan2(0.25_5).txt", "wb") as fp:   #Pickling
  pickle.dump(mng_full_clustering, fp)

with eps=0.25 and min_samples=5 we have:

In [111]:
mng_dbscan_labels=np.unique(mng_full_clustering.labels_)
for label in mng_dbscan_labels:
    print('cluster:',label)
    bb=np.where(mng_full_clustering.labels_==label)
    print('number of members:',len(list(bb[0])))
    if (label!=-1) :
        target=0
        ind=list(bb[0]) 
#        print(ind)
        for j in ind:
#            print(tweet_mng_cleaned_full[j])
            if 'manag' in  tweet_mng_cleaned_full[j]:
                target +=1
        print('number of target:',target)
        
        
    print('****'*5)

cluster: -1
number of members: 96427
********************
cluster: 0
number of members: 14
number of target: 0
********************
cluster: 1
number of members: 11
number of target: 0
********************
cluster: 2
number of members: 17
number of target: 0
********************
cluster: 3
number of members: 33
number of target: 0
********************
cluster: 4
number of members: 8
number of target: 0
********************
cluster: 5
number of members: 239
number of target: 0
********************
cluster: 6
number of members: 9
number of target: 0
********************
cluster: 7
number of members: 5
number of target: 0
********************
cluster: 8
number of members: 19
number of target: 0
********************
cluster: 9
number of members: 15
number of target: 0
********************
cluster: 10
number of members: 24
number of target: 0
********************
cluster: 11
number of members: 7
number of target: 0
********************
cluster: 12
number of members: 17
number of target: 0
*

with eps=10 and min_samples=5 we have

In [69]:
np.unique(mng_full_clustering.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,
       84, 85, 86], dtype=int64)

In [77]:
with open("tweet_mng_cleaned_full_clustering_dbscan2(10_5).txt", "rb") as fp:   #Pickling
  mng_full_clustering = pickle.load(fp)

In [78]:
mng_dbscan_labels=np.unique(mng_full_clustering.labels_)
type(mng_dbscan_labels)

numpy.ndarray

clusters and thier number of members for eps=10 , min_samples=5

In [93]:
for label in mng_dbscan_labels:
    print('cluster:',label)
    bb=np.where(mng_full_clustering.labels_==label)
    print('number of members:',len(list(bb[0])))
    if (label!=-1) and (label!=0):
        target=0
        ind=list(bb[0]) 
        print(ind)
        for j in ind:
            print(tweet_mng_cleaned_full[j])
            if 'playoff' in  tweet_mng_cleaned_full[j]:
                target +=1
        print('number of target:',target)
        
        
    print('****'*5)

cluster: -1
number of members: 22799
********************
cluster: 0
number of members: 76196
********************
cluster: 1
number of members: 7
[251, 1928, 16166, 24510, 46136, 60993, 98263]
rt charg playoff like
yeah manag playoff
rt hat playoffmod
anoth playoff pope
playoff
alreadi playoff
playoff form
number of target: 7
********************
cluster: 2
number of members: 15
[418, 20460, 25703, 27378, 29741, 31681, 47928, 49452, 51856, 60631, 75927, 79369, 93893, 94367, 95807]
motormouth fan pleas follow motor mouthuk share motor stori us
motormouth fan pleas follow motor mouthuk share motor stori us
motormouth fan pleas follow motor mouthuk share motor stori us
motormouth fan pleas follow motor mouthuk share motor stori us
motormouth fan pleas follow motor mouthuk share motor stori us
motormouth fan pleas follow motor mouthuk share motor stori us
motormouth fan pleas follow motor mouthuk share motor stori us
motormouth fan pleas follow motor mouthuk share motor stori us
motormout

with eps=3 and min_samples=10 we have

In [74]:
with open("tweet_mng_cleaned_full_clustering_dbscan.txt", "rb") as fp:   # Unpickling
  mng_full_clustering = pickle.load(fp) 

In [75]:
mng_dbscan_labels=np.unique(mng_full_clustering.labels_)
type(mng_dbscan_labels)


numpy.ndarray

clusters and thier number of members for eps=3 , min_samples=10

In [76]:
for label in mng_dbscan_labels:
     print('cluster:',label)
     bb=np.where(mng_full_clustering.labels_==label)
     print('number of members:',len(list(bb[0])))
     print('****'*5)
         

cluster: -1
number of members: 97276
********************
cluster: 0
number of members: 162
********************
cluster: 1
number of members: 11
********************
cluster: 2
number of members: 17
********************
cluster: 3
number of members: 33
********************
cluster: 4
number of members: 245
********************
cluster: 5
number of members: 12
********************
cluster: 6
number of members: 23
********************
cluster: 7
number of members: 15
********************
cluster: 8
number of members: 25
********************
cluster: 9
number of members: 17
********************
cluster: 10
number of members: 15
********************
cluster: 11
number of members: 12
********************
cluster: 12
number of members: 10
********************
cluster: 13
number of members: 54
********************
cluster: 14
number of members: 13
********************
cluster: 15
number of members: 19
********************
cluster: 16
number of members: 34
********************
cluster: 17
num

since almost all of data in one cluster, it does not give us good results